In [ ]:
import serial
import numpy as np
from numpy.linalg import eig, inv
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [ ]:
def get_data(port="/dev/ttyUSB0", baud=115200):
    ser = serial.Serial(port, baud, timeout=0.1)
    #ser.readline()
    while True:
        line = str(ser.readline(), 'utf8').replace("\x00"," ").strip()
        #print(line)
        if line:
            try:
                vals= [float(v) for v in line.split()]
                if len(vals)>=9:
                    yield vals
            except:pass

In [ ]:
def ls_ellipsoid(x, y, z, inverse=1):
    """least squares fit to a 3D-ellipsoid
    Ax^2 + By^2 + Cz^2 +  Dxy +  Exz +  Fyz +  Gx +  Hy +  Iz  = 1

    Note that sometimes it is expressed as a solution to
    Ax^2 + By^2 + Cz^2 + 2Dxy + 2Exz + 2Fyz + 2Gx + 2Hy + 2Iz  = 1
    where the last six terms have a factor of 2 in them
    This is in anticipation of forming a matrix with the polynomial coefficients.
    Those terms with factors of 2 are all off diagonal elements.  These contribute
    two terms when multiplied out (symmetric) so would need to be divided by two

    http://juddzone.com/ALGORITHMS/least_squares_3D_ellipsoid.html"""

    # change xx from vector of length N to Nx1 matrix so we can use hstack
    x, y, z = [np.array(a)[:, None] for a in (x, y, z)]

    #  Ax^2 + By^2 + Cz^2 +  Dxy +  Exz +  Fyz +  Gx +  Hy +  Iz = 1
    J = np.hstack((x * x, y * y, z * z, x * y, x * z, y * z, x, y, z))
    K = np.ones_like(x)  # column of ones

    JT = J.transpose()
    JTJ = np.dot(JT, J)
    InvJTJ = inv(JTJ)
    C = np.dot(InvJTJ, np.dot(JT, K))
    C = np.append(C, -1)

    print("coefficients", np.array_str(C, precision=3, max_line_width=999))

    # convert the polynomial form of the 3D-ellipsoid to parameters
    # center, axes, and transformation matrix
    # vec is the vector whose elements are the polynomial
    # coefficients A..J
    # returns (center, axes, rotation matrix)

    # Algebraic form: X.T * Amat * X --> polynomial form

    A = (
        np.array(
            [
                [2 * C[0], C[3], C[4], C[6]],
                [C[3], 2 * C[1], C[5], C[7]],
                [C[4], C[5], 2 * C[2], C[8]],
                [C[6], C[7], C[8], 2 * C[9]],
            ]
        )
        / 2
    )

    # See B.Bartoni, Preprint SMU-HEP-10-14 Multi-dimensional Ellipsoidal Fitting
    # equation 20 for the following method for finding the center
    # https://www.physics.smu.edu/~scalise/SMUpreprints/SMU-HEP-10-14.pdf
    A3 = A[0:3, 0:3]
    A3inv = inv(A3)
    ofs = C[6:9] / 2
    center = -np.dot(A3inv, ofs)
    print("center", np.array_str(center, precision=3))

    # Center the ellipsoid at the origin
    Tofs = np.eye(4)
    Tofs[3, 0:3] = center
    R = np.dot(Tofs, np.dot(A, Tofs.T))

    R3 = R[0:3, 0:3]
    R3test = R3 / R3[0, 0]
    # print("R3test", R3test)
    s1 = -R[3, 3]
    R3S = R3 / s1
    (el, ec) = eig(R3S)

    recip = 1.0 / np.abs(el)
    axes = np.sqrt(recip)
    print("axes", np.array_str(axes, precision=3))

    rot = inv(ec)  # inverse is actually the transpose here
    print("rotation", np.array_str(rot, precision=3))

    M = np.eye(4)
    M[0:3, 0:3] = rot.T.dot(np.diag(axes)).dot(rot)
    M[0:3, 3] = center

    return inv(M) if inverse else M

In [ ]:
def matrix(o=0, n=2000,m=0):
    for i, vals in enumerate(get_data()):
        if m and np.linalg.norm(np.array(vals[o:o+3]))>m: continue
        if i == 0:
            data = np.array(vals)
        else:
            data = np.vstack([data, vals])
        if i % 20 == 0:
            # print(data.transpose())
            d = data.transpose()
            x, y, z = d[o], d[o + 1], d[o + 2]
            plt.plot(x, y, "r.")
            plt.plot(y, z, "g.")
            plt.plot(z, x, "b.")
            plt.gca().set_aspect("equal", adjustable="box")
            clear_output(wait=True)
            display(plt.gcf())
            print(len(data))
        if i >= n:
            break

    clear_output(wait=True)

    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    ax.scatter(x, y, z, marker=".")
    plt.show()

    d = data.transpose()
    C = ls_ellipsoid(x, y, z)
    print("matrix")
    for i in range(3):
        print("{",', '.join(map(str,C[i])),"},")
    return C

## Accelerometer Calibration

Run the following cell an turn the sensor slowly to cover the entier sphere with data points.

C&P the resulting calibration matrix into Arduino sketch in variable `acal`.

In [ ]:
A=matrix()

## Magnetometer Calibration

Run the following cell an turn the sensor slowly to cover the entier sphere with data points.

C&P the resulting calibration matrix into Arduino sketch in variable `mcal`.

In [ ]:
M=matrix(6)

In [ ]:
G=None
for i, vals in enumerate(get_data()):
    if i>100: break
    g=np.array(vals[3:6])
    if G is None: G=g
    G=G+0.01*(g-G)
    print(i,G)
    clear_output(wait=True)
print("{",", ".join(map(str,G)),"}")

## Test Calibration

In [ ]:
# from ahrs.filters import Madgwick,SAAM
from time import monotonic


def tostr(a):
    return "[" + ", ".join(f"{v:6.3f}" for v in a) + "]"


deg = 180 / np.pi
rad = np.pi / 180

madgwick = Madgwick(Dt=0.068,gain=0.1)
q = np.array([1.0, 0.0, 0.0, 0.0])
saam=SAAM()
t = monotonic()
for i, v in enumerate(get_data()):
    print("v", tostr(v))
    a = np.array(v[0:3])
    a = np.matmul(A[:3], np.array([*a, 1]))
    print("a", tostr(a), f"{np.linalg.norm(a):.3f}")
    g = np.array(v[3:6])
    g=g-G
    print("g", tostr(g))
    b = np.array(v[6:9])
    b = np.matmul(M[:3], np.array([*b, 1]))
    print("b", tostr(b), f"|b| {np.linalg.norm(b):.3f}    a.m {np.dot(b,a):.3f}")

    u = monotonic()
    #madgwick.updateIMU(q=q,acc=a,gyr=g*rad)
    madgwick.updateMARG(q=q, acc=a, gyr=g *rad, mag=b)
    #print("Q", q, u - t)
    w, x, y, z = q
    roll = -np.arctan2(2.0 * (z * y + w * x), 1.0 - 2.0 * (x * x + y * y))
    pitch = np.arcsin(2.0 * (y * w - z * x))
    heading = np.arctan2(2.0 * (z * w + x * y), -1.0 + 2.0 * (w * w + x * x))
    heading = (540 - heading * deg) % 360
    print(f"MADGWICK heading {heading:5.1f}°   pitch {pitch*deg:4.1f}°   roll {roll*deg:4.1f}°")

    qs=saam.estimate(acc=a,mag=b)
    w, x, y, z = qs
    roll = -np.arctan2(2.0 * (z * y + w * x), 1.0 - 2.0 * (x * x + y * y))
    pitch = np.arcsin(2.0 * (y * w - z * x))
    heading = np.arctan2(2.0 * (z * w + x * y), -1.0 + 2.0 * (w * w + x * x))
    heading = (540 - heading * deg) % 360
    print(f"SAAM     heading {heading:5.1f}°   pitch {pitch*deg:4.1f}°   roll {roll*deg:4.1f}°")
    
    t = u

    a[1] *= -1
    b[1] *= -1
    a /= np.linalg.norm(a)

    p = -np.arcsin(a[0])
    r = np.arcsin(a[1] / np.cos(p))
    x = np.cos(p) * b[0] + np.sin(p) * b[2]
    y = np.sin(r) * np.sin(p) * b[0] + np.cos(r) * b[1] - np.sin(r) * np.cos(p) * b[2]
    h = (540 - np.arctan2(y, x) * deg) % 360
    print(f"DIRECT   heading {h:5.1f}°   pitch {p*deg:4.1f}°   roll {r*deg:4.1f}°")

    clear_output(wait=True)